In [1]:
import pandas as pd
import os

In [2]:
interim_dir = "../data/interim"
processed_dir = "../data/processed"

data_model_file = os.path.join(processed_dir, "data_model_output.csv")

In [3]:
data_model_df = pd.read_csv(data_model_file)
data_model_df.head()
data_model_df = data_model_df[(data_model_df['is_completed'] == 1) & (data_model_df['record_type_synthetic'] == 0)]

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_27812\3153797618.py:1: DtypeWarning: Columns (1,11,12,13,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,52,53,57,63,68,72,75,76,78,80,81,84,90,91,93,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,137,144,145,146,149,150,151,156,161,163,171,173,179,188,189,190,193,198,202,203,208,210,211,214,215,231,233,234,235,236,237,238,240,242,243,244,245,246,256,257,262,267,275,276,277,280,283,287,288,289,291,292,311,325,327,329,330,332,344,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,394,398,400,403,404,405,427,428) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_df = pd.read_csv(data_model_file)


In [4]:
working_df = data_model_df[['marketsegment_label', 'passenger_type_label', 'main_mode_label', 'access_mode_label', 
                            'egress_mode_label', 'origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude', 'transit_boarding_latitude', 'transit_boarding_longitude',
                            'transit_alighting_latitude', 'transit_alighting_longitude']]

In [5]:
working_df.head()

,marketsegment_label,passenger_type_label,main_mode_label,access_mode_label,egress_mode_label,origin_latitude,origin_longitude,destination_latitude,destination_longitude,transit_boarding_latitude,transit_boarding_longitude,transit_alighting_latitude,transit_alighting_longitude
0,EMPLOYEE,NaN,DROVE_ALONE_AND_PARKED,NaN,NaN,32.816714,-117.176898,32.732627,-117.204064,NaN,NaN,NaN,NaN
1,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.639943,-117.085774,32.733783,-117.193315,32.715857,-117.168938,32.732133,-117.197756
2,EMPLOYEE,NaN,DROVE_ALONE_AND_PARKED,NaN,NaN,32.743009,-117.131699,32.732627,-117.204064,NaN,NaN,NaN,NaN
3,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.706752,-117.148810,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010
4,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.678108,-117.099196,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010


In [6]:
len(working_df)

5276

In [7]:
working_df = working_df[
    working_df[['transit_boarding_latitude', 'transit_boarding_longitude',
                'transit_alighting_latitude', 'transit_alighting_longitude']].notna().any(axis=1)
]


In [8]:
working_df.head()

,marketsegment_label,passenger_type_label,main_mode_label,access_mode_label,egress_mode_label,origin_latitude,origin_longitude,destination_latitude,destination_longitude,transit_boarding_latitude,transit_boarding_longitude,transit_alighting_latitude,transit_alighting_longitude
1,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.639943,-117.085774,32.733783,-117.193315,32.715857,-117.168938,32.732133,-117.197756
3,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.706752,-117.148810,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010
4,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.678108,-117.099196,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010
6,EMPLOYEE,NaN,AIRPORT_FLYER_SHUTTLE,WALK,NaN,32.755747,-117.197638,32.732627,-117.204064,32.755309,-117.199526,32.731527,-117.202270
7,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.690364,-117.174227,32.733783,-117.193315,32.715832,-117.154063,32.732133,-117.197756


In [9]:
len(working_df)

450

In [14]:
working_df['access_mode_label'].value_counts()

access_mode_label
WALK                                    196
DROVE_ALONE_AND_PARKED                   76
DROPPED_OFF_BY_FAMILY_FRIEND             60
UBER_LYFT                                22
OTHER_PUBLIC_TRANSIT                     17
DROVE_WITH_OTHERS_AND_PARKED              9
CAR_SERVICE_BLACK_LIMO                    7
OTHER                                     6
RODE_WITH_OTHER_TRAVELERS_AND_PARKED      3
BICYCLE_PERSONAL_NON_ELECTRIC             1
TAXI                                      1
Name: count, dtype: int64

In [11]:
from geopy.distance import geodesic

# Filtering departing passengers and calculating distance between transit_boarding and origin
working_df.loc[(working_df['passenger_type_label'] == 'DEPARTING') | (working_df['marketsegment_label'] == 'EMPLOYEE'), 'distance'] = working_df.apply(
    lambda row: geodesic(
        (row['origin_latitude'], row['origin_longitude']), 
        (row['transit_boarding_latitude'], row['transit_boarding_longitude'])
    ).miles, axis=1
)

# Filtering arriving passengers and calculating distance between transit_alighting and destination
working_df.loc[working_df['passenger_type_label'] == 'ARRIVING', 'distance'] = working_df.apply(
    lambda row: geodesic(
        (row['destination_latitude'], row['destination_longitude']), 
        (row['transit_alighting_latitude'], row['transit_alighting_longitude'])
    ).miles, axis=1
)

In [12]:
working_df['distance']

1        7.131336
3        0.696556
4        4.120447
6        0.114004
7        2.111942
          ...    
5425     2.362784
5426     3.815009
5427     9.103463
5433    16.389507
5439     1.319569
Name: distance, Length: 450, dtype: float64

In [34]:
len(working_df[working_df['distance']<1])

103

In [36]:
distance_less_than_a_mile_df = working_df[working_df['distance']<0.5]
distance_less_than_a_mile_df['access_mode_label'].value_counts()

access_mode_label
WALK                             55
DROPPED_OFF_BY_FAMILY_FRIEND      5
OTHER_PUBLIC_TRANSIT              2
DROVE_ALONE_AND_PARKED            1
BICYCLE_PERSONAL_NON_ELECTRIC     1
CAR_SERVICE_BLACK_LIMO            1
DROVE_WITH_OTHERS_AND_PARKED      1
UBER_LYFT                         1
Name: count, dtype: int64

In [29]:
working_df.head()

,marketsegment_label,passenger_type_label,main_mode_label,access_mode_label,egress_mode_label,origin_latitude,origin_longitude,destination_latitude,destination_longitude,transit_boarding_latitude,transit_boarding_longitude,transit_alighting_latitude,transit_alighting_longitude,distance
1,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.639943,-117.085774,32.733783,-117.193315,32.715857,-117.168938,32.732133,-117.197756,7.131336
3,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.706752,-117.148810,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010,0.696556
4,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.678108,-117.099196,32.732627,-117.204064,32.715832,-117.154063,32.731419,-117.203010,4.120447
6,EMPLOYEE,NaN,AIRPORT_FLYER_SHUTTLE,WALK,NaN,32.755747,-117.197638,32.732627,-117.204064,32.755309,-117.199526,32.731527,-117.202270,0.114004
7,EMPLOYEE,NaN,MTS_ROUTE_992,WALK,NaN,32.690364,-117.174227,32.733783,-117.193315,32.715832,-117.154063,32.732133,-117.197756,2.111942


In [ ]:
mean_distance = working_df.groupby(
    ['marketsegment_label', 'access_mode_label']
).agg(
    avg_distance_miles=('distance', 'mean'),
    num_respondents=('distance', 'count')
).reset_index()

mean_distance.head(20)

,marketsegment_label,egress_mode_label,avg_distance_miles,num_respondents
0,PASSENGER,OTHER_SHARED_VAN,12.640424,1
1,PASSENGER,PICKED_UP_BY_FAMILY_FRIEND,14.982186,14
2,PASSENGER,TAXI,8.343507,2
3,PASSENGER,UBER_LYFT,8.902334,2
4,PASSENGER,WALK,8.505221,31
